In [1]:
import requests
import pandas as pd
import numpy as np
import time

from datetime import datetime, timedelta
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
from IPython.display import clear_output


DECIMAL = 18
OS_API_KEY = 'OS_API_KEY'
OS_EVENTS_URL = 'https://api.opensea.io/api/v1/events'

In [2]:
def get_opensea_events(begin_date, end_date, event_type, limit=300):
    payload = {
        'occurred_before': str(end_date.timestamp()),
        'occurred_after': str(begin_date.timestamp()),
        'limit': limit,
        'event_type': event_type,
    }
    headers = {
        'Accept': 'application/json',
        'x-api-key': OS_API_KEY,
    }
    s = requests.Session()
    retries = Retry(total=2, backoff_factor=0.1)
    s.mount('https://', HTTPAdapter(max_retries=retries))
    return s.get(
        OS_EVENTS_URL, 
        params=payload, 
        headers=headers,
        timeout=5,
    )

def process_raw_opensea_events(events, 
                               event_types=('created', 'successful', 'cancelled'),
                               payment_tokens=('ETH',)):
    events_ = []
    for event in events:
                
        # Skip transfers and offers
        if event['event_type'] not in event_types:
            continue
        # Skip bundles
        if event['asset'] is None:
            continue
        # Only keep Ethereum NFTs (no Polygon or free airdrops)
        if event['payment_token'] is None or event['payment_token']['symbol'] not in payment_tokens:
            continue
        # Remove private listings
        if event['is_private']:
            continue
                    
        try:
            events_.append({
                'id': event['id'],
                'collection_slug': event['collection_slug'], 
                'contract_name': event['asset']['asset_contract']['name'],
                'contract_address': event['asset']['asset_contract']['address'],
                'asset_name': event['asset']['name'],
                'token_id': event['asset']['token_id'],
                'event_type': event['event_type'], 
                'created_date': datetime.strptime(event['created_date'], '%Y-%m-%dT%H:%M:%S.%f'), 
                'starting_price': float(event['starting_price']) / 10 ** DECIMAL if event['starting_price'] else np.nan, 
                'ending_price': float(event['ending_price']) / 10 ** DECIMAL if event['ending_price'] else np.nan, 
                'total_price': float(event['total_price']) / 10 ** DECIMAL if event['total_price'] else np.nan,
                'payment_token': event['payment_token']['symbol'] if event['payment_token'] is not None else None,
                'seller': event['seller']['address'] if event['seller'] is not None else None,
                'buyer': event['winner_account']['address'] if event['winner_account'] is not None else None,
            })
        except Exception as e:
            pass
    
    try:
        return pd.DataFrame(events_).set_index('id').sort_values('created_date')
    except Exception as e:
        empty_df = pd.DataFrame({
            'id': pd.Series([], dtype='int'),
            'collection_slug': pd.Series([], dtype='str'), 
            'contract_name': pd.Series([], dtype='str'),
            'contract_address': pd.Series([], dtype='str'),
            'asset_name': pd.Series([], dtype='str'),
            'token_id': pd.Series([], dtype='str'),
            'event_type': pd.Series([], dtype='str'), 
            'created_date': pd.Series([], dtype='datetime64[ns]'), 
            'starting_price': pd.Series([], dtype='float'), 
            'ending_price': pd.Series([], dtype='float'), 
            'total_price': pd.Series([], dtype='float'),
            'payment_token': pd.Series([], dtype='str'),
            'seller': pd.Series([], dtype='str'),
            'buyer': pd.Series([], dtype='str'),
        }) \
        .set_index('id')
        return empty_df
    
def get_all_opensea_events(begin_date, end_date, event_types=('created', 'successful', 'cancelled')):
    try:
        events_dfs = []
        for event_type in event_types:
            r = get_opensea_events(begin_date, end_date, event_type=event_type).json()
            events_dfs.append(process_raw_opensea_events(r['asset_events'] if 'asset_events' in r else []))
        events_df = pd.concat(events_dfs).sort_values('created_date')
        return events_df
    except:
        return pd.DataFrame()
    
def add_new_opensea_events(
    events_df, 
    event_types=('created', 'successful', 'cancelled'),
    event_type_time_offset={'successful': 60}):
    
    end_date = datetime.now()
    try:
        events_dfs = []
        for event_type in event_types:
            begin_date = events_df[events_df.event_type == event_type].created_date.max()
            if event_type in event_type_time_offset:
                begin_date -= timedelta(seconds=event_type_time_offset[event_type])
            r = get_opensea_events(begin_date, end_date, event_type=event_type).json()
            events_dfs.append(process_raw_opensea_events(r['asset_events'] if 'asset_events' in r else []))
            
        events_df_ = pd.concat([events_df] + events_dfs).sort_values('created_date', ascending=False)
        events_df_ = events_df_[~events_df_.index.duplicated(keep='first')]
        return events_df_
    
    except:
        return events_df

In [3]:
import logging

for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(
    filename='logs/nft_analytics.log', 
    filemode='w', 
    level=logging.INFO,
    format='%(asctime)s.%(msecs)03d %(levelname)s - %(funcName)s: %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
)

In [4]:
end_date = datetime.now()
begin_date = end_date - timedelta(seconds=6000)

events_df = get_all_opensea_events(begin_date, end_date)
events_df

,collection_slug,contract_name,contract_address,asset_name,token_id,event_type,created_date,starting_price,ending_price,total_price,payment_token,seller,buyer
id,,,,,,,,,,,,,
7014909050,moonrunnersnft,Moonrunners,0x1485297e942ce64e0870ece60179dfda34b4c625,Moonrunners #2421,2421,cancelled,2022-06-26 20:37:42.378797,0.5500,0.5500,NaN,ETH,0xd3eabb661771911f87c50cf105bda74468c75b01,None
7014909063,karafuru-gachapon,Karafuru Gachapon,0xf220db48f0d3ca8a9833e0353e7497dbceae7ac6,Karafuru Gachapon #617,617,cancelled,2022-06-26 20:37:42.470933,0.6170,0.6170,NaN,ETH,0xc0ebdcb0591b7d8631a26e5e269dd3a667415a7f,None
7014909074,rebelsbynight,Rebels,0x7deda0afe6df3da6a85a87b371f8b464c30c6803,Rebel #3983,3983,cancelled,2022-06-26 20:37:42.546169,0.4200,0.4200,NaN,ETH,0xbd0dde4175d8e854306a09de4c184392b33a6d9c,None
7014909751,bbyc-bored-bits-yacht-club,Bored Bits Yacht Club,0x5f1cbe84d44e292fe3eae51b87f34bbdc8f04fc8,#3883,3883,cancelled,2022-06-26 20:37:52.385635,0.0350,0.0350,NaN,ETH,0x91bcee36abea706b8d769e5440eb62922463a9a9,None
7014909757,bbyc-bored-bits-yacht-club,Bored Bits Yacht Club,0x5f1cbe84d44e292fe3eae51b87f34bbdc8f04fc8,#3883,3883,cancelled,2022-06-26 20:37:52.452356,0.0444,0.0444,NaN,ETH,0x91bcee36abea706b8d769e5440eb62922463a9a9,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7014947767,thepoopstars,OpenSea Collection,0x495f947276749ce646f68ac8c248420045cb7b5e,Mark Zuckerberg,7488697810909665580977852589134212357394418851...,created,2022-06-26 20:46:57.448754,0.0506,0.0506,NaN,ETH,0xa590870e16288831ce9ebcea873396b37bc7565d,None
7014947769,woobirds,Woobirds,0xffddd1bd34dbe1368def6b27d2893768c10175e3,#594,594,created,2022-06-26 20:46:57.503537,0.0130,0.0130,NaN,ETH,0xc9ee81d9aa01a3d069011d5cb8bb3c7461a10ea9,None
7014947775,deluxe-dogs,OpenSea Collection,0x495f947276749ce646f68ac8c248420045cb7b5e,Deluxe Dog #9810,6216850407611543085923444463529850010150962483...,created,2022-06-26 20:46:57.732973,0.0100,0.0100,NaN,ETH,0x89722058fb4c5ae0d943b67cbd1014a4bc873116,None


In [5]:
# Plotly

import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go

from plotly.subplots import make_subplots

pio.templates.default = 'plotly_dark'

# Dash

from jupyter_dash import JupyterDash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output

In [6]:
app = JupyterDash(__name__)
app.layout = html.Div(children=[
    dcc.Dropdown(
        id='collection-slug-dropdown',
    ),
    dcc.Graph(
        id='nft-analytics-graph',
        config={'scrollZoom': True},
    ),
    dcc.Interval(
        id='interval-component',
        interval=1000,
        n_intervals=0,
    ),
])

In [7]:
from tsmoothie.smoother import KalmanSmoother


@app.callback(Output('collection-slug-dropdown', 'options'),
              Input('interval-component', 'n_intervals'))
def collection_slug_dropdown(n_intervals):
    recent_buy_events_df = events_df[(events_df.created_date >= (datetime.utcnow() - timedelta(seconds=60 * 15))) & 
                                     (events_df.event_type == 'successful')]
    recent_buy_stats_df = recent_buy_events_df \
        .groupby('collection_slug') \
        .agg({'total_price': ['count', 'mean', 'sum']}) \
        .sort_values(('total_price', 'count'), ascending=False) \
        .reset_index()
    
    return [{'label': f'{row.collection_slug[0]} (num_trades={row[("total_price", "count")]}, volume={row[("total_price", "sum")]}, avg_price={row[("total_price", "mean")]})', 'value': row.collection_slug[0]} for _, row in recent_buy_stats_df.iterrows()]

@app.callback(Output('nft-analytics-graph', 'figure'),
              [Input('collection-slug-dropdown', 'value'), 
               Input('interval-component', 'n_intervals')])
def nft_analytics_graph(collection_slug, n_intervals):
    events_df_ = events_df[(events_df.collection_slug == collection_slug) & (events_df.event_type.isin(['created', 'successful']))]

    buy_events_df = events_df_[events_df_.event_type == 'successful']
    sell_events_df = events_df_[events_df_.event_type == 'created']
    
    # --------------------------------- Aggregate ---------------------------------

    bars_df = events_df_ \
    .resample('1min', on='created_date') \
    .agg({
        'total_price': ['count', 'sum', 'min', 'max'], 
        'starting_price': ['count', 'sum', 'min', 'max'],
    })

    bars_df.columns = bars_df.columns.map('{0[0]}_{0[1]}'.format)
    bars_df.reset_index(inplace=True)
    bars_df.rename(columns={
        'total_price_count': 'buy_volume_count',
        'total_price_sum': 'buy_volume_eth',
        'total_price_min': 'buy_min_eth',
        'total_price_max': 'buy_max_eth',
        'starting_price_count': 'sell_volume_count',
        'starting_price_sum': 'sell_volume_eth',
        'starting_price_min': 'sell_min_eth',
        'starting_price_max': 'sell_max_eth',
    }, inplace=True)
    bars_df = bars_df.set_index('created_date')

    # --------------------------------- Generate Attributes ---------------------------------

    bars_df['avg_price'] = buy_events_df.set_index('created_date') \
    .rolling(10).median() \
    .reset_index() \
    .resample('1min', on='created_date').mean()['total_price'] \
    .fillna(method='ffill')

    sell_events_join_floor_price_df = sell_events_df.assign(
        created_date_bar=sell_events_df['created_date'].apply(lambda x: x.replace(second=0, microsecond=0))
    ) \
    .set_index('created_date_bar') \
    .join(bars_df[['avg_price']]) \
    .fillna(method='ffill')

    above_avg_thresh = 1.05 # pct above current average price

    bars_df['near_avg_sell_volume_count'] = sell_events_join_floor_price_df \
    .resample('1min', on='created_date') \
    .apply(lambda r: sum(r['starting_price'] < r['avg_price'] * above_avg_thresh))
    
    # Smoothed volumes
    
    smoother = KalmanSmoother(component='level_trend', 
                              component_noise={'level': 0.1, 'trend': 0.1})
    
    smoother.smooth(bars_df.fillna(0)['buy_volume_count'].ewm(alpha=0.1).mean().fillna(0))
    bars_df['smoothed_buy_volume_count'] = smoother.smooth_data[0]
    bars_df['smoothed_buy_volume_count'] = bars_df['smoothed_buy_volume_count'].clip(0)
    
    smoother.smooth(bars_df.fillna(0)['near_avg_sell_volume_count'].ewm(alpha=0.1).mean().fillna(0))
    bars_df['smoothed_near_avg_sell_volume_count'] = smoother.smooth_data[0]
    bars_df['smoothed_near_avg_sell_volume_count'] = bars_df['smoothed_near_avg_sell_volume_count'].clip(0)
    
    # ================================= Plot =================================
    
    fig = make_subplots(
        rows=3, 
        shared_xaxes=True,
        row_heights=[0.6, 0.2, 0.2], 
        vertical_spacing=0,
    )
    
    # ------------ Price ------------ #
    
    fig.add_trace(
        go.Scatter(
            x=buy_events_df.created_date, 
            y=buy_events_df.total_price, 
            name='Sale',
            mode='markers',
            marker=dict(color='green', opacity=0.4,
                        line=dict(width=0.5, color='green')),
        ),
        row=1, col=1,
    )
    
    fig.add_trace(
        go.Scatter(
            x=sell_events_df.created_date, 
            y=sell_events_df.starting_price, 
            name='List',
            mode='markers',
            marker=dict(color='red', opacity=0.4,
                        line=dict(width=0.5, color='red')),
        ),
        row=1, col=1,
    )

    fig.add_trace(
        go.Scatter(
            x=bars_df.index, 
            y=bars_df.avg_price, 
            name='Average Price',
            line=dict(color='white'),
        ),
        row=1, col=1,
    )
    
    fig.layout['yaxis1']['range'] = (bars_df.avg_price.min() * 0.95, bars_df.avg_price.max() * 1.05)
    fig.layout['yaxis1']['autorange'] = False
    
    # ------------ Raw Supply vs. Demand Pct ------------ #
    
    fig.add_trace(
        go.Bar(
            x=bars_df.index,
            y=bars_df.buy_volume_count,
            name='Buy Volume',
            marker_color='green',
        ),
        row=2, col=1,
    )

    fig.add_trace(
        go.Bar(
            x=bars_df.index,
            y=bars_df.near_avg_sell_volume_count,
            name='Sell Volume',
            marker_color='red',
        ),
        row=2, col=1,
    )
    
    # ------------ Smoothed Supply vs. Demand Pct ------------ #
    
    fig.add_trace(
        go.Scatter(
            x=bars_df.index, 
            y=bars_df.smoothed_buy_volume_count, 
            name='Smoothed Buy Volume',
            marker=dict(color='green', opacity=0.4,
                        line=dict(width=0.5, color='green')),
        ),
        row=3, col=1,
    )

    fig.add_trace(
        go.Scatter(
            x=bars_df.index, 
            y=bars_df.smoothed_near_avg_sell_volume_count, 
            name='Smoothed Sell Volume',
            marker=dict(color='red', opacity=0.4,
                        line=dict(width=0.5, color='red')),
        ),
        row=3, col=1,
    )
    
    # ====================================================================================
    
    fig.update_layout(
        height=1000, 
        dragmode='pan',
        uirevision=collection_slug,
        hovermode='x unified',
        spikedistance=-1,
    )
    
    fig.update_yaxes(showline=True, showspikes=True, spikemode='across', spikesnap='cursor', spikedash='solid')
    fig.update_xaxes(showline=True, showspikes=True, spikemode='across', spikesnap='cursor', spikedash='solid')
    
    fig.add_vline(x=datetime.utcnow(), line=dict(color='orange', dash='dot'))
    
    return fig

In [8]:
app.run_server(mode='external', debug=False)

Dash app running on http://127.0.0.1:8050/


In [ ]:
%%capture

while True:
    events_df = add_new_opensea_events(events_df)
    events_df = events_df[events_df.created_date >= (datetime.utcnow() - timedelta(hours=4))]
    time.sleep(1.5)